In [1]:
!pip install torch torchvision

In [2]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 32.0 MB/s eta 0:00:00


In [3]:
!pip install onnxruntime-training

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 MB 4.0 MB/s eta 0:00:00


In [4]:
import torch
import torchvision
import onnx # Import onnx
import os
from torchvision import models
import torch.nn as nn
from torchvision.models import MobileNet_V2_Weights

In [6]:
class CelebAMobileNet(nn.Module):
    """
    A custom MobileNetV2 model for the CelebA dataset with a frozen feature extractor
    and a customizable classifier head.

    Args:
        num_classes (int): The number of output classes for the classifier head.
    """

    def __init__(self, num_classes=4):
        super(CelebAMobileNet, self).__init__()

        # Load the pre-trained MobileNetV2 model
        self.model = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V2)

        # Freeze the feature extractor
        for param in self.model.features.parameters():
            param.requires_grad = False

        # Replace the classifier head with a new fully connected layer
        self.model.classifier[1] = nn.Linear(self.model.last_channel, num_classes)

        # Initialize the new classifier head
        self._initialize_classifier()

    def _initialize_classifier(self):
        """Initialize the classifier head parameters."""
        nn.init.kaiming_uniform_(self.model.classifier[1].weight)
        nn.init.zeros_(self.model.classifier[1].bias)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, channels, height, width].

        Returns:
            torch.Tensor: Output tensor of shape [batch_size, num_classes].
        """
        return self.model(x)

In [10]:
model = CelebAMobileNet(num_classes=4)

# Load the state dictionary with weights_only=True
state_dict = torch.load('/content/model_state_dict.pth', map_location=torch.device('cpu'), weights_only=True)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

<All keys matched successfully>

In [11]:
# Export the model to ONNX.
model_name = "mobilenetv2"
# Create the 'training_artifacts' directory if it doesn't exist
os.makedirs("training_artifacts", exist_ok=True)
torch.onnx.export(model, torch.randn(1, 3, 224, 224),
                  f"training_artifacts/{model_name}.onnx",
                  input_names=["input"], output_names=["output"],
                  dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}})

In [13]:
from onnxruntime.training import artifacts

# Load the onnx model.
onnx_model = onnx.load(f"training_artifacts/{model_name}.onnx")

In [17]:
for node in onnx_model.graph.node:
    # Print the node's name and its input names directly
    print(node.name, node.input)

/model/features/features.0/features.0.0/Conv ['input', 'onnx::Conv_538', 'onnx::Conv_539']
/model/features/features.0/features.0.2/Constant []
/model/features/features.0/features.0.2/Constant_1 []
/model/features/features.0/features.0.2/Clip ['/model/features/features.0/features.0.0/Conv_output_0', '/model/features/features.0/features.0.2/Constant_output_0', '/model/features/features.0/features.0.2/Constant_1_output_0']
/model/features/features.1/conv/conv.0/conv.0.0/Conv ['/model/features/features.0/features.0.2/Clip_output_0', 'onnx::Conv_541', 'onnx::Conv_542']
/model/features/features.1/conv/conv.0/conv.0.2/Constant []
/model/features/features.1/conv/conv.0/conv.0.2/Constant_1 []
/model/features/features.1/conv/conv.0/conv.0.2/Clip ['/model/features/features.1/conv/conv.0/conv.0.0/Conv_output_0', '/model/features/features.1/conv/conv.0/conv.0.2/Constant_output_0', '/model/features/features.1/conv/conv.0/conv.0.2/Constant_1_output_0']
/model/features/features.1/conv/conv.1/Conv ['/m

In [18]:
requires_grad = ["model.classifier.1.weight", "model.classifier.1.bias"]
frozen_params = [
   param.name
   for param in onnx_model.graph.initializer
   if param.name not in requires_grad
]


# Generate the training artifacts.
artifacts.generate_artifacts(
   onnx_model,
   requires_grad=requires_grad,
   frozen_params=frozen_params,
   loss=artifacts.LossType.CrossEntropyLoss,
   optimizer=artifacts.OptimType.AdamW,
   artifact_directory="training_artifacts"
)

In [19]:
import shutil

shutil.make_archive("training_artifacts", 'zip', "training_artifacts")

'/content/training_artifacts.zip'

In [20]:
from google.colab import files

files.download("training_artifacts.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>